# Initialization

In [1]:
# IMPORTS
import azure.functions as func
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

#from config import * 
from datetime import datetime, timedelta
from newspaper import Article
# from newspaper import Config as npConfig
import pandas as pd
import numpy as np 
import re
import numbers

import logging
logging.getLogger('azure').propagate = False
logging.getLogger('urllib3').propagate = False
import warnings
warnings.filterwarnings("ignore")

In [2]:
# CONFIG

# GOOGLENEWS 
searchword = 'LCH SA'
start_dt='01/01/2021'
end_dt='01/31/2021'

# Twitter Keys
api_key = 'Oc9II7RV8McqVCXjU9MybOvnP' 
api_secret_key = 'KNGE1JX1xIKbwUoucdr2s3ZBwQBY6SYBAHY3qEpGWreQANaNZR'
access_token = '1347130231908421632-Wqvur6Qc1Emo3uj1U2Meyz2uJ6X45R'
access_token_secret = 'zZ7NEnJh7F3LnPUBjTjkhCUJhFpCuGndcmw2rGvUWrGSL'

# Azure Cognitive Service: Text Analytics Keys
cog_endpoint = 'https://eocjmcognitiveservicesconnection.cognitiveservices.azure.com/'
cog_key = '1b79872142ca4bf482381e946bbb6dc2'

# DF Columns
columns = ['API', 'Link', 'Date', 'Title', 'Article', 'Summary', 
               'Title_Sentiment', 'Title_Positv', 
               'Title_Neutral', 'Title_Negativ',
               'Article_Sentiment', 'Article_Positv', 
               'Article_Neutral', 'Article_Negativ'
              ] 
            
root_path = r'C:\Users\Johannes\Documents\Data Science\eoc_newsscraper_'
file_format = '.csv'

In [3]:
# FUNCTIONS

# Message and DF 
def get_init_message(searchword):
    """"""
    # Date 
    date = datetime.today().strftime('%Y-%m-%d')
    time = datetime.today().strftime('%H:%M')
    message = date+' '+time +'\n'
    
    message += 'WebScraper: ' + searchword+ '\n\n'
    
    return message 
def get_init_df():
    """"""
    return pd.DataFrame(columns=columns)
def save_csv(dff, searchword):
    """"""
    date = datetime.today().strftime('%Y%m%d')
    path = root_path +  searchword+'_'+ date + file_format
    dff.to_csv(path, index=False)

# Azure     
def authenticate_azure_textanalytics_client():
    ta_credential = AzureKeyCredential(cog_key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=cog_endpoint, 
            credential=ta_credential)
    return text_analytics_client
def getAzureSentiment(text, client):
    """"""
    documents = [text[:5120]] # limit to 5120 characters. This is the azure data limit for text analytics
    response = client.analyze_sentiment(documents = documents)[0]
    document_sentiment = response.sentiment
    score_positive = response.confidence_scores.positive
    score_neutral = response.confidence_scores.neutral
    score_negative = response.confidence_scores.negative
    return document_sentiment, score_positive, score_neutral, score_negative
def get_newssearch_client():
    """"""
    # Import Libraries
    from azure.cognitiveservices.search.newssearch import NewsSearchClient
    from msrest.authentication import CognitiveServicesCredentials
    
    # Credentials
    subscription_key = "5521444b482f4dd6a800e85fdfb1c636"
    ENDPOINT = "https://api.bing.microsoft.com/v7.0/search?q=microsoft+devices"
    
    # Client
    bing_client = NewsSearchClient(
            endpoint=ENDPOINT,
            credentials=CognitiveServicesCredentials(subscription_key)
        )
    
    return bing_client

# Add news to df
def articles_to_df(urls, dff, api):
    # Import News API
    from newspaper import Article

    # Authenticate connection to azure text analytics service
    client = authenticate_azure_textanalytics_client()

    for url in urls:
        # Initialize 
        title, text, summary='','',''
        date = ''
        try:
            # Download Article
            article = Article(url)
            article.download()
            article.parse()
            article.nlp()

            title = article.title
            text = article.text
            summary = article.summary
            date = article.publish_date
        except:
            print('Article could not be downloaded. ' + url)
            #continue

        try:
            # Call Sentiment Analizer
            title_sentiment, title_pos, title_neu, title_neg = getAzureSentiment(title, client)
            text_sentiment, text_pos, text_neu, text_neg = getAzureSentiment(text, client)
        except:
            print('Azure Sentiment could not be called. ' + url)

        dff = dff.append({'API':api,
                        'Link':url, 
                        'Date':date, 
                        'Title':title, 
                        'Article':text, 
                        'Summary':summary, 
                        'Title_Sentiment':title_sentiment,
                        'Title_Positv':title_pos, 
                        'Title_Neutral':title_neu, 
                        'Title_Negativ':title_neg, 
                        'Article_Sentiment':text_sentiment,
                        'Article_Positv':text_pos, 
                        'Article_Neutral':text_neu, 
                        'Article_Negativ':text_neg}, 
                       ignore_index=True) 
    
    return dff
def tweets_to_df(dff, tweets_data):
    """"""
    

    # Authenticate connection to azure text analytics service
    client = authenticate_azure_textanalytics_client()

    for i in range(len(tweets_data[0])):
        # Initialize 
        # tweets_data : [texts, dates, users]
        title = tweets_data[0][i]
        date = tweets_data[1][i]
        link = 'User: '+ tweets_data[2][i]
        
        # Verify that tweet is actually about LCH SA (TO BE IMPROVED)
        if ((False==(('LCH' in title)or('lch sa' in title))
             or('Light combat Helic' in title))):
            continue

        try:
            # Call Sentiment Analizer
            title_sentiment, title_pos, title_neu, title_neg = getAzureSentiment(title, client)
        except:
            print('Twitter: Azure Sentiment could not be called.')

        dff = dff.append({'API':'Twitter',
                        'Link':link, 
                        'Date':date, 
                        'Title':title, 
                        'Article':'', 
                        'Summary':'', 
                        'Title_Sentiment':title_sentiment,
                        'Title_Positv':title_pos, 
                        'Title_Neutral':title_neu, 
                        'Title_Negativ':title_neg, 
                        'Article_Sentiment':'',
                        'Article_Positv':'', 
                        'Article_Neutral':'', 
                        'Article_Negativ':''}, 
                       ignore_index=True) 
    
    return dff
def ft_news_to_df(ft_dict, dff):
    """"""
    ft_url_base = 'https://www.ft.com'
    
    for i in range(len(ft_dict.get('Link'))):
        # Get Article 
        title = ft_dict.get('Title')[i]
        date = ft_dict.get('Date')[i][:10]
        link = ft_url_base + ft_dict.get('Link')[i]
        
        # Authenticate connection to azure text analytics service
        client = authenticate_azure_textanalytics_client()
        try:
          # Call Sentiment Analizer
          title_sentiment, title_pos, title_neu, title_neg = getAzureSentiment(title, client)
        except:
            print('Twitter: Azure Sentiment could not be called.')

        dff = dff.append({'API':'FinancialTimes',
                            'Link':link, 
                            'Date':date, 
                            'Title':title, 
                            'Article':'', 
                            'Summary':'', 
                            'Title_Sentiment':title_sentiment,
                            'Title_Positv':title_pos, 
                            'Title_Neutral':title_neu, 
                            'Title_Negativ':title_neg, 
                            'Article_Sentiment':'',
                            'Article_Positv':'', 
                            'Article_Neutral':'', 
                            'Article_Negativ':''}, 
                           ignore_index=True) 
    
    return dff
def bing_news_to_df(results, dff):
    """"""
    
    # Authenticate connection to azure text analytics service
    ta_client = authenticate_azure_textanalytics_client()
    
    for result in results:
        name = result.get("name")
        url = result.get("url")
        date = result.get("date_published")[:10]
        description = result.get("description")
        provider = result.get("provider")[0].get("name")
        
        row = {}
        
        try:
            # Download Article
            article = Article(result.get("url"))
            article.download()
            article.parse()
            article.nlp()

            title = article.title
            text = article.text
            summary = article.summary
            
            # Add Articles to Df 
            row = {'API':'Bing',
                    'Link':url, 
                    'Date':date, 
                    'Title':title, 
                    'Article':text, 
                    'Summary':summary, 
                    'Title_Sentiment':getAzureSentiment(title, ta_client)[0],
                    'Title_Positv':getAzureSentiment(title, ta_client)[1], 
                    'Title_Neutral':getAzureSentiment(title, ta_client)[2],
                    'Title_Negativ':getAzureSentiment(title, ta_client)[3], 
                    'Article_Sentiment':getAzureSentiment(text, ta_client)[0],
                    'Article_Positv':getAzureSentiment(text, ta_client)[1], 
                    'Article_Neutral':getAzureSentiment(text, ta_client)[2], 
                    'Article_Negativ':getAzureSentiment(text, ta_client)[3]}
        except:
            #print('Article could not be downloaded. ' + url)
            row = {'API':'Bing',
                    'Link':url, 
                    'Date':date, 
                    'Title':name, 
                    'Article':description, 
                    'Summary':'', 
                    'Title_Sentiment':getAzureSentiment(name, ta_client)[0],
                    'Title_Positv':getAzureSentiment(name, ta_client)[1], 
                    'Title_Neutral':getAzureSentiment(name, ta_client)[2],
                    'Title_Negativ':getAzureSentiment(name, ta_client)[3], 
                    'Article_Sentiment':getAzureSentiment(description, ta_client)[0],
                    'Article_Positv':getAzureSentiment(description, ta_client)[1], 
                    'Article_Neutral':getAzureSentiment(description, ta_client)[2], 
                    'Article_Negativ':getAzureSentiment(description, ta_client)[3]}
            #continue
        
        # Add row to dff
        dff = dff.append(row, ignore_index=True)
        
    return dff  
    
# Fetch News via APIs
def add_google_news(dff, searchword):
    """"""
    #Import API
    from GoogleNews import GoogleNews

    # Max and Min Date of Search
    start_dt = (datetime.today() - timedelta(days=30)).strftime('%m/%d/%Y')
    end_dt = datetime.today().strftime('%m/%d/%Y')

    # Fetch News 
    googlenews=GoogleNews(start=start_dt,end=end_dt, lang='en') # language english
    googlenews.search(searchword)
    result=googlenews.result()

    # List of news urls 
    urls = [news.get('link') for news in result]
    
    dff = articles_to_df(urls, dff, api='Google')
    
    return dff
def add_scrapy_news(dff, searchword):
    """"""
    # Import
    import scrapy
    from scrapy.crawler import CrawlerProcess
    import logging
    logging.getLogger('scrapy').propagate = False

    # Create the Spider class
    class news_Spider(scrapy.Spider):
        name = "News_Spider"
        global scrapy_dict
        global ft_dict

        # start_requests method
        def start_requests(self):
            urls = ['https://www.risk.net/search/articles?query=lch&sort_by=date&date-filter=7+day&topics%5B%5D=249061&organisations%5B%5D=255101',
                       'https://www.ft.com/stream/3da687f2-11a7-4e28-a18d-17b77aa18460']
            for url in urls:
                yield scrapy.Request(url = url, callback = self.parse_front)

        # First parsing method
        def parse_front(self, response):
            risk_url = 'https://www.risk.net'
            ft_url = 'https://www.ft.com'
            if(response.url[:len(risk_url)]==risk_url):
                # Extract Link
                link = response.xpath('//h5[@class="article-title"]/a/@href')
                link_extract = link.extract()
                scrapy_dict['risk']=link_extract
            elif(response.url[:len(ft_url)]==ft_url):
                link = response.xpath('//*[@id="stream"]/div[1]/ul/li/div[2]/div/div/div/div[2]/a/@href') 
                link_extract = link.extract()
                scrapy_dict['ft']= link_extract 
                ft_dict['Link'] = link_extract
                # Extract Title
                title = response.xpath('//*[@id="stream"]/div[1]/ul/li/div[2]/div/div/div/div[2]/a/text()') 
                ft_dict['Title'] = title.extract()
                # Extract Date
                date = response.xpath('//div[@class="stream-card__date"]/time/@datetime') 
                ft_dict['Date'] = date.extract()

    scrapy_dict = dict()
    ft_dict = dict()

    # Run the Spider
    process = CrawlerProcess()
    process.crawl(news_Spider)
    process.start()

    # Build Link List
    risk_url_base = 'http://www.risk.net'

    # Add Risk.Net News 
    urls = [url if (url[:4]=='http') else risk_url_base+url for url in scrapy_dict.get('risk')]
    dff = articles_to_df(urls, dff, api='Risk.Net')
    
    # Add FT News 
    dff = ft_news_to_df(ft_dict, dff)

    return dff    
def add_twitter_news(dff, searchword):
    """"""
    #Import
    import tweepy
    
    # Consumer keys and access tokens, used for OAuth
    consumer_key = api_key
    consumer_secret = api_secret_key

    # OAuth process, using the keys and tokens
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    # Creation of the actual interface, using authentication
    api = tweepy.API(auth)
    
    # Fetch Tweets 
    searchterm = searchword[1:-1] + ' -filter:retweets' # Exclude Retweets
    tweets = api.search(q=searchterm, lang="en" ) #  , result_type='recent'
    
    def clearnUpTweet(txt):
        """"""
        txt = re.sub(r'@', '', txt) # Remove mentions # [A-Za-z0-9_]+
        txt = re.sub(r'#', ' ', txt) # Remove hashtags
        txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt) # Remove urls
        txt = re.sub(r'&gt;', ' is greater than ', txt) # 
        txt = re.sub(r'&gt;', ' is smaller than ', txt) # 
        txt = re.sub(r'\n', ' ', txt) #
        txt = re.sub(r'w/', 'with', txt) # 
        return txt
    
    # Tweets Data
    texts = [clearnUpTweet(tweet.text) for tweet in tweets]
    dates = [tweet.created_at.date() for tweet in tweets]
    users = [tweet.user.name for tweet in tweets] # Bug: This is not the actual user that posted it
    tweets_data = [texts, dates, users]
    
    # Add Tweets to DF
    dff = tweets_to_df(dff, tweets_data)
    
    return dff
def add_bing_news(dff, searchword):
    """"""
    # Authenticate connection to azure bing news search service
    bing_client = get_newssearch_client()
    
    # Fetch News
    news_result = bing_client.news.search(query=searchword,
            #market="en-uk", NOT WORKING 
            freshness="Week",
            #setLang:"en", NOT WORKING
            sort_by="Date"
        )
    
    # Add Bing news to df  
    dff =  bing_news_to_df(news_result.as_dict().get("value"), dff)
    
    return dff


#  Additional Features
def verify_articles(dff, searchword):
    """"""
    # Authenticate Azure text analysis client 
    ta_client = authenticate_azure_textanalytics_client()
    
    def language_detection_example(text, client):
        try:
            documents = [text]
            response = client.detect_language(documents = documents)[0]
            language = response.primary_language.name
            return language
        except Exception as err:
            return 'Unknown'
        
    verified_col = []
    for row in dff.itertuples():
        verified = '' 

        #Check Language of Article
        text = row.Article if len(row.Article)>25 else row.Title
        language = language_detection_example(row.Article, ta_client)
        if (language != 'English'):
            verified += 'Detected language is ' + language + '. '

        # Check if Article is actually about searchword
        if (searchword.lower() not in row.Title.lower()):
            verified += searchword + ' does not appear in the title.'

        # Is verified
        verified_col.append(verified)

    dff['Verified'] = verified_col
    dff['Verified_Flag'] = 'True' if (len(verified_col)<1) else 'False'
    
    return dff
def add_keyword_search(dff):
    """"""
    # Keyword Search
    def find_keywords(article):
        """"""
        from fuzzysearch import find_near_matches
        keywords = ['fraud', 'claim', 'incident', 'bankruptcy', 'fine', 'bankrupt',
                    'Fraud', 'Claim', 'Incident', 'Bankruptcy', 'Fine', 'Bankrupt']
        words = ''
        for keyword in keywords:
            count = len(find_near_matches(keyword, article, max_l_dist=0))
            if(count>0):
                words += str(count)+'x '+keyword+', '
        if(len(words)>1):
            words = words[:-2]
        else:
            words = '-'
        return words
    
    dff['Found_Keywords'] = dff.Article.apply(find_keywords)
    return dff
def add_ner_search(dff):
    """"""
    
    # Authenticate connection to azure text analytics service
    text_analytics_client = authenticate_azure_textanalytics_client()
    
    # NER Search
    def get_ner_string(article):
        """"""
        try:
            article = article[:5120] # Max length for article # should be split in multisearch
            document = [article]

            response = text_analytics_client.recognize_entities(document , language="en")
            result = [doc for doc in response if not doc.is_error]

            ner_entities = []
            ner_categories, ner_scores = [], []

            for entity in result[0].entities:
                if (entity.category in ['Organization']):#,'Skill']):
                    ner_entities.append(entity.text) 
                    ner_categories.append(entity.category)
                    #ner_scores += entity.confidence_score

            return str(list(set(ner_entities)))[1:-1]
        except:
            return '-'
    
    # Add Ner column 
    dff['NER'] = dff.Article.apply(get_ner_string)
    
    return dff 
def add_topic_search(dff):
    """"""
    # Authenticate Azure text analysis client 
    text_analytics_client = authenticate_azure_textanalytics_client()
    
    def get_article_topics(article):
        """"""
        article = article[:5120]
        document = [article]

        try:
            response = text_analytics_client.extract_key_phrases(document)
            result = [doc for doc in response if not doc.is_error]
            topics = result[0].key_phrases
        except: 
            topics = []
        topics = str(topics)[1:-1]
        return topics
    
    # Add Ner column 
    dff['Topics'] = dff.Article.apply(get_article_topics)
    
    return dff 

# Text Message 
def get_sentiment_message(dff, message):
    """"""
    # Overall Sentiment  
    sentiment_list = [*dff.Title_Sentiment] + [*dff.Article_Sentiment]
    sentiment = max(set(sentiment_list), key = sentiment_list.count) 
    message += 'The overall sentiment of the articles was '+sentiment + '.\n\n'
    # Mean sentiment
    def str_mean(x):
        x = np.array([val for val in x if isinstance(val, numbers.Number)])
        return str(round(x.mean(),2))

    message += 'Mean Sentiment: Title ('\
                                + str_mean(dff.Title_Positv)+', '\
                                + str_mean(dff.Title_Neutral)+', '\
                                + str_mean(dff.Title_Negativ)+'); '\
                            +'Article ('+str_mean(dff.Article_Positv)+', '\
                                        +str_mean(dff.Article_Positv)+', '\
                                        +str_mean(dff.Article_Negativ)+').+\n\n'
    # Negativ Articles
    # Articles with a negativ score >0.33
    neg_th =  0.33
    neg_df = dff[(dff.Title_Negativ>neg_th)|(dff.Title_Negativ>neg_th)]
    neg_count=len(neg_df) 
    if(neg_count==1):
        message += 'There is '+str(neg_count) + ' article with a negativ sentiment:\n'
    elif(neg_count>1):
        message += 'There are '+str(neg_count) + ' articles with a negativ sentiment:\n'
    for row in neg_df.itertuples():
         message += '\t'+row.Title + ' ['+row.Title_Sentiment+' title ('\
                                            +str(row.Title_Positv)+', '\
                                            +str(row.Title_Neutral)+', '\
                                            +str(row.Title_Negativ)+'); '\
                                        +row.Article_Sentiment+' article ('\
                                            +str(row.Article_Positv)+', '\
                                            +str(row.Article_Neutral)+', '\
                                            +str(row.Article_Negativ)+')]+\n\n'
    
    return message

# Main

In [10]:
# Main
def main(searchword):
    
    # Get Message and DF
    message = get_init_message(searchword)
    df = get_init_df()
    
    # Add News 
    #df = add_google_news(df, searchword) # API can be blocked via Azure
    #df = add_scrapy_news(df, searchword) # For now only for LCH SA
    #df = add_bing_news(df, searchword)
    #df = add_twitter_news(df, searchword)
    
    # Add specific Articles
    urls = ['https://www.vanityfair.com/news/2020/06/bankrupt-hertz-is-a-pandemic-zombie',
            #'https://www.forbes.com/sites/jackkelly/2020/05/23/hertz-files-for-bankruptcy-after-16000-employees-were-let-go-and-ceo-made-over-9-million/'
            'https://www.nbcnews.com/politics/meet-the-press/hertz-jcpenny-j-crew-join-list-businesses-filing-bankruptcy-n1213961',
            'https://edition.cnn.com/2020/05/22/business/hertz-bankruptcy/index.html'
           ]  
    df = articles_to_df(urls, df, 'Urls')
    
    
    # Verification Step: Are news actually about searchword
    df = verify_articles(df, searchword)
    
    # Find Keywords like Fraud, etc.
    df = add_keyword_search(df)
    
    # Ner and Topic Search 
    df = add_ner_search(df)
    df = add_topic_search(df)
    
    #Get Overall Sentiment
    message = get_sentiment_message(df, message)
    
    #Save results 
    save_csv(df, searchword)
    
    return message, df

import logging
logging.getLogger('scrapy').propagate = False
 
searchword='LCH SA'
#searchword="EUREX"
#searchword='HERTZ'
message, df = main(searchword)
# import pprint as pp
# pp.pprint(message)

In [12]:
df

,API,Link,Date,Title,Article,Summary,Title_Sentiment,Title_Positv,Title_Neutral,Title_Negativ,Article_Sentiment,Article_Positv,Article_Neutral,Article_Negativ,Verified,Verified_Flag,Found_Keywords,NER,Topics
0,Urls,https://www.vanityfair.com/news/2020/06/bankru...,None,“This Is Just F--king Unbelievable!”: Bankrupt...,The pandemic and its economic consequences hav...,"Then, in a class of absurdity by itself, is wh...",negative,0.00,0.03,0.97,mixed,0.1,0.07,0.83,Detected language is Unknown.,False,"5x bankruptcy, 9x bankrupt","'Boeing', 'Herbalife', 'Revco Drug Stores', 'W...","'Hertz stock', 'Hertz shareholders', 'Hertz bo..."
1,Urls,https://www.nbcnews.com/politics/meet-the-pres...,2020-05-24 13:06:00+00:00,"Hertz, JCPenney, JCrew join list of businesses...",WASHINGTON — When the history of the COVID-19 ...,"In the space of just two weeks, some of the be...",neutral,0.01,0.74,0.25,mixed,0.2,0.08,0.72,,False,"1x claim, 2x bankruptcy, 4x bankrupt","'JC Penney', 'Bar Louie', 'Neiman Marcus', 'Ar...","'employees', 'mortar locations', 'economy', 'c..."
2,Urls,https://edition.cnn.com/2020/05/22/business/he...,2020-05-22 00:00:00,Hertz files for bankruptcy,"The company has been renting cars since 1918, ...","By declaring bankruptcy, Hertz says it intends...",neutral,0.02,0.56,0.42,mixed,0.1,0.07,0.83,Detected language is Unknown.,False,"6x bankruptcy, 7x bankrupt","'Avis Budget Group', 'Firefly', 'Diamond Offsh...","'brands Hertz', 'John Hertz', 'year', 'Hertz H..."


In [9]:
df.Link[17]

'https://www.msn.com/fr-ca/actualites/r%C3%A9gion%20de%20trois-rivi%C3%A8res/lart-de-se-faire-un-pr%C3%A9nom/ar-BB1dEiS9'

In [7]:
message

'2021-02-15 11:36\nWebScraper: LCH SA\n\nThe overall sentiment of the articles was neutral.\n\nMean Sentiment: Title (0.1, 0.71, 0.19); Article (0.27, 0.27, 0.55).+\n\nThere are 3 articles with a negativ sentiment:\n\tLCH.Clearnet raises margin on Italian debt [negative title (0.01, 0.18, 0.81);  article (, , )]+\n\n\tClearing war flares up in Europe as Euro CCP cuts fees [negative title (0.0, 0.0, 1.0);  article (, , )]+\n\n\tL’art de se faire un prénom [negative title (0.27, 0.08, 0.65); mixed article (0.3, 0.14, 0.56)]+\n\n'

In [49]:
save_csv(df, searchword)

In [20]:
dfc = df.copy()

# Concept

__DF__  
api link date title article summary media | titlesentiment (positive, neutral negative) | articlesentiment (positive, neutral negative)

tweet date account | sentiment (positive, neutral negative)

- __Enhance Sentiment Analysis__ 

- Keywords
    - Add in overall sentiment: bad sentiment + keyword
- Fix Twitter
- Add Links to scrapy search urls
    - ['https://www.vindobona.org/article/vienna-stock-exchange-new-calculation-technology-increases-quality',
 'https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/']
    - Scrapy Last in Search and always activated
- Change Scrapy FT searchsite
- FT Subscription?
- Make Scrapy repeatable
- Suppress Scrapy Log
- Backtest with Hertz
- Change NER and Topicsearch max limit to multisearch

## Backtesting Example

Hertz filed bankruptcy on the 22nd of May 2020.

## Temp

In [67]:
path = r'C:\Users\Johannes\Documents\Data Science\eoc_newsscraper_LCH SA_20210215.csv'
tmp=pd.read_csv(path)

In [72]:
tmp.Article[1]

'Since 2016, LSEG has been upgrading London Clearing House’s (LCH’s) core technology stack and this year the cash equities business, EquityClear Ltd., went live with LSEG Technology’s Millennium Clearing and Millennium Risk (MCCP) platform on March 9, during a period that saw extreme volatility. The technology revamp has sealed LSEG’s AFTA win for the best new technology category for trading and risk over the last 12 months.\n\nIn its initial go-live week at EquityClear, the MCCP platform processed over 8 million trades, which is more than two-and-a-half times the historical daily average. Powered by graphical processing units, the platform can now process risk computations faster than its predecessor and enables EquityClear to gauge exposure and margin changes on a trade-by-trade basis, with Historical Value at Risk (HVaR) running a full revaluation.\n\nWhile margin runs are typically triggered at end-of-day, intraday, or on an ad hoc basis, it has now been extended to trigger at 10-m

In [61]:
tmp.Date = pd.to_datetime(tmp.Date.str[:10]).dt.strftime('%d/%m/%Y')

In [73]:
path = r'C:\Users\Johannes\Documents\Data Science\eoc_newsscraper_LCH SA_20210215.csv'
tmp.to_csv(path)

# Feature Tests

In [4]:
# Initialize DF
columns = ['API', 'Link', 'Date', 'Title', 'Article', 'Summary', 'Topics',
           'Found_Keywords', 'Overall_Sentiment', 'Verified', 
           'Title_Sentiment', 'Title_Positv', 'Title_Neutral', 'Title_Negativ',
           'Article_Sentiment', 'Article_Positv', 'Article_Neutral', 'Article_Negativ'
          ]
dff = pd.DataFrame(columns=columns)
searchword='EUREX'  
dff = add_bing_news(dff, searchword)
#dff = add_scrapy_news(dff, searchword)
message = ''

In [7]:
dff = df.copy()

## Article Verification

In [107]:
def verify_articles(dff, searchword):
    """"""
    # Authenticate Azure text analysis client 
    ta_client = authenticate_azure_textanalytics_client()
    
    def language_detection_example(text, client):
        try:
            documents = [text]
            response = client.detect_language(documents = documents)[0]
            language = response.primary_language.name
            return language
        except Exception as err:
            return 'Unknown'
        
    verified_col = []
    for row in dff.itertuples():
        verified = '' 

        #Check Language of Article
        text = row.Article if len(row.Article)>25 else row.Title
        language = language_detection_example(row.Article, ta_client)
        if (language != 'English'):
            verified += 'Detected language is ' + language + '. '

        # Check if Article is actually about searchword
        if (searchword not in row.Title):
            verified += searchword + ' does not appear in the title.'

        # Is verified
        verified_col.append(verified)

    dff['Verified'] = verified_col
    dff['Verified_Flag'] = 'True' if (len(verified_col)<2) else 'False'
    
    return dff 
dff = verify_articles(dff, searchword)

## Topic Search

In [120]:
def add_topic_search(dff):
    """"""
    # Authenticate Azure text analysis client 
    text_analytics_client = authenticate_azure_textanalytics_client()
    
    def get_article_topics(article):
        """"""
        article = article[:5120]
        document = [article]

        try:
            response = text_analytics_client.extract_key_phrases(document)
            result = [doc for doc in response if not doc.is_error]
            topics = result[0].key_phrases
        except: 
            topics = []
        topics = str(topics)[1:-1]
        return topics
    
    # Add Ner column 
    dff['Topics'] = dff.Article.apply(get_article_topics)
    
    return dff 

In [123]:
def get_article_topics(article):
    """"""
    article = article[:5120]
    document = [article]
    
    try:
        response = text_analytics_client.extract_key_phrases(document)
        result = [doc for doc in response if not doc.is_error]
        topics = result[0].key_phrases
    except: 
        topics = []
    topics = str(topics)[1:-1]
    return topics

In [9]:
dff = add_topic_search(dff)
dff.head()

,API,Link,Date,Title,Article,Summary,Title_Sentiment,Title_Positv,Title_Neutral,Title_Negativ,Article_Sentiment,Article_Positv,Article_Neutral,Article_Negativ,Verified,Verified_Flag,Found_Keywords,NER,Topics
0,Bing,https://www.leaprate.com/forex/market-news/for...,2021-02-12,"Forex Weekly Look Back: StoneX revenue up 1,20...",Multi-asset broker Exness will be taking part ...,,neutral,0.00,1.00,0.00,neutral,0.02,0.97,0.01,EUREX does not appear in the title.,False,,Bing,"'iFEX virtual expo', 'financial virtual expo',..."
1,Bing,https://www.modernreaders.com/news/2021/02/12/...,2021-02-12,Contrasting Deutsche Börse (DBOEY) and Its Rivals,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,Analyst RatingsGet Deutsche Börse alerts:This ...,neutral,0.04,0.78,0.18,mixed,0.22,0.23,0.55,EUREX does not appear in the title.,False,,Bing,"'Deutsche Börse rivals', 'Deutsche Börse Compe..."
2,Bing,https://theenterpriseleader.com/2021/02/12/deu...,2021-02-12,Deutsche Börse (DBOEY) versus Its Peers Head t...,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,DividendsGet Deutsche Börse alerts:Deutsche Bö...,neutral,0.01,0.96,0.03,mixed,0.29,0.19,0.52,EUREX does not appear in the title.,False,,Bing,"'Deutsche Börse peers', 'Deutsche Börse Compet..."
3,Bing,https://theenterpriseleader.com/2021/02/12/fin...,2021-02-12,Financial Comparison: Deutsche Börse (DBOEY) a...,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,DividendsGet Deutsche Börse alerts:Deutsche Bö...,neutral,0.06,0.89,0.05,mixed,0.31,0.20,0.49,EUREX does not appear in the title.,False,,Bing,"'Deutsche Börse peers', 'Deutsche Börse Compet..."
4,Bing,https://apnews.com/press-release/pr-newswire/t...,2021-02-12,Fenics GO named OTC Trading Platform of the Year,"LONDON, Feb. 11, 2021 /PRNewswire/ -- Fenics G...","LONDON, Feb. 11, 2021 /PRNewswire/ -- Fenics G...",neutral,0.01,0.98,0.01,mixed,0.26,0.09,0.65,Detected language is Unknown. EUREX does not a...,False,,Bing,"'Fenics division of BGC Partners', 'resiliency..."


## NER Extraction

https://azuresdkdocs.blob.core.windows.net/$web/python/azure-ai-textanalytics/1.0.0b1/index.html

In [9]:
# Authenticate connection to azure text analytics service
text_analytics_client = authenticate_azure_textanalytics_client()

In [76]:
def add_ner_search(dff):
    """"""
    
    # Authenticate connection to azure text analytics service
    text_analytics_client = authenticate_azure_textanalytics_client()
    
    # NER Search
    def get_ner_string(article):
        """"""
        article = article[:5120] # Max length for article # should be split in multisearch
        document = [article]

        response = text_analytics_client.recognize_entities(document , language="en")
        result = [doc for doc in response if not doc.is_error]

        ner_entities = []
        ner_categories, ner_scores = [], []

        for entity in result[0].entities:
            if (entity.category in ['Organization']):#,'Skill']):
                ner_entities.append(entity.text) 
                ner_categories.append(entity.category)
                #ner_scores += entity.confidence_score

        return str(list(set(ner_entities)))[1:-1]
    
    # Add Ner column 
    dff['NER'] = dff.Article.apply(get_ner_string)
    
    return dff 
df['NER'] = add_ner_search(df)

,API,Link,Date,Title,Article,Summary,Title_Sentiment,Title_Positv,Title_Neutral,Title_Negativ,Article_Sentiment,Article_Positv,Article_Neutral,Article_Negativ,Found_Keywords,NER
0,Bing,https://www.leaprate.com/forex/market-news/for...,2021-02-12,"Forex Weekly Look Back: StoneX revenue up 1,20...",Multi-asset broker Exness will be taking part ...,,neutral,0.00,1.00,0.00,neutral,0.02,0.97,0.01,,'Exness'
1,Bing,https://www.modernreaders.com/news/2021/02/12/...,2021-02-12,Contrasting Deutsche Börse (DBOEY) and Its Rivals,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,Analyst RatingsGet Deutsche Börse alerts:This ...,neutral,0.04,0.78,0.18,mixed,0.22,0.23,0.55,,"'Xetra', 'GSF', 'EEX', 'MarketBeat.com', 'Qont..."
2,Bing,https://theenterpriseleader.com/2021/02/12/deu...,2021-02-12,Deutsche Börse (DBOEY) versus Its Peers Head t...,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,DividendsGet Deutsche Börse alerts:Deutsche Bö...,neutral,0.01,0.96,0.03,mixed,0.29,0.19,0.52,,"'Xetra', 'GSF', 'EEX', 'MarketBeat.com', 'Qont..."
3,Bing,https://theenterpriseleader.com/2021/02/12/fin...,2021-02-12,Financial Comparison: Deutsche Börse (DBOEY) a...,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,DividendsGet Deutsche Börse alerts:Deutsche Bö...,neutral,0.06,0.89,0.05,mixed,0.31,0.20,0.49,,"'Deutsche Börse', 'MarketBeat', 'Data', 'Clear..."
4,Bing,https://apnews.com/press-release/pr-newswire/t...,2021-02-12,Fenics GO named OTC Trading Platform of the Year,"LONDON, Feb. 11, 2021 /PRNewswire/ -- Fenics G...","LONDON, Feb. 11, 2021 /PRNewswire/ -- Fenics G...",neutral,0.01,0.98,0.01,mixed,0.26,0.09,0.65,,"'Citadel Securities', 'BGC Trader', 'BGC Group..."
5,Bing,https://www.modernreaders.com/news/2021/02/12/...,2021-02-12,Deutsche Börse (DBOEY) and The Competition Cri...,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,Given Deutsche Börse’s peers stronger consensu...,negative,0.00,0.01,0.99,mixed,0.33,0.21,0.46,,"'Investment', 'Xetra', 'GSF', 'EEX', 'Qontigo'..."
6,Bing,https://www.financemagnates.com/institutional-...,2021-02-10,Clearstream Reports €15.4 Trillion in Assets U...,Settlement volumes at Deutsche Boerse AG’s Cle...,Settlement volumes at Deutsche Boerse AG’s Cle...,neutral,0.01,0.89,0.10,mixed,0.27,0.11,0.62,,"'Xetra', 'Deutsche Börse Group', 'Deutsche Boe..."
7,Bing,https://www.vindobona.org/article/vienna-stock...,2021-02-09,Vienna Stock Exchange: New Calculation Technol...,The Vienna Stock Exchange (Wiener Börse AG) ha...,,neutral,0.01,0.99,0.00,positive,0.50,0.48,0.02,,"'Wiener Börse AG', 'Vienna Stock Exchange'"
8,Bing,https://www.leaprate.com/financial-services/ex...,2021-02-08,Eurex traded contracts fall 13%MoM in January ...,Eurex Repo posted drops 25.2% MoM rise in GC P...,,neutral,0.00,0.96,0.04,neutral,0.01,0.87,0.12,,"'Eurex Repo', 'GC'"


In [12]:
for row in dfc.itertuples():
    article = row.Article
    print('x')

    article = article[:5120] # Max length for article # should be split in multisearch
    document = [article]

    response = text_analytics_client.recognize_entities(document , language="en")
    result = [doc for doc in response if not doc.is_error]

    ner_entities = []
    ner_categories, ner_scores = [], []

    for entity in result[0].entities:
        if (entity.category in ['Organization']):#,'Skill']):
            ner_entities.append(entity.text) 
            ner_categories.append(entity.category)
            #ner_scores += entity.confidence_score

    print(str(list(set(ner_entities)))[1:-1])

x
'Eurex'
x


IndexError: list index out of range

In [57]:
documents = [
    "Microsoft was founded by Bill Gates and Paul Allen. \
     Redmond is a city in King County, Washington, United States, located 15 miles east of Seattle.",
    "Jeff bought three dozen eggs because there was a 50% discount."
]
documents = [df.Article[1]]

response = text_analytics_client.recognize_entities(documents, language="en")
result = [doc for doc in response if not doc.is_error]

for doc in result:
    print('XXX')
    for entity in doc.entities:
        print("Entity: ", entity.text, "\tCategory: ", entity.category,
              "\tConfidence Score: ", entity.confidence_score)

XXX
Entity:  Deutsche Börse 	Category:  Organization 	Confidence Score:  0.98
Entity:  one 	Category:  Quantity 	Confidence Score:  0.8
Entity:  23 	Category:  Quantity 	Confidence Score:  0.8
Entity:  Security 	Category:  Skill 	Confidence Score:  0.8
Entity:  commodity 	Category:  Skill 	Confidence Score:  0.8
Entity:  Deutsche Börse 	Category:  Organization 	Confidence Score:  0.97
Entity:  businesses 	Category:  Skill 	Confidence Score:  0.8
Entity:  analyst 	Category:  PersonType 	Confidence Score:  0.86
Entity:  analyst 	Category:  Skill 	Confidence Score:  0.8
Entity:  valuation 	Category:  Skill 	Confidence Score:  0.8
Entity:  Analyst 	Category:  Skill 	Confidence Score:  0.8
Entity:  Deutsche Börse 	Category:  Organization 	Confidence Score:  0.95
Entity:  Deutsche Börse 	Category:  Organization 	Confidence Score:  0.96
Entity:  MarketBeat.com 	Category:  Organization 	Confidence Score:  0.9
Entity:  Deutsche Börse 	Category:  Organization 	Confidence Score:  0.78
Entity:  0 

Entity:  Deutsche Börse 	Category:  Organization 	Confidence Score:  0.95
Entity:  MarketBeat 	Category:  Organization 	Confidence Score:  0.65
Entity:  MarketBeat.com 	Category:  Organization 	Confidence Score:  0.9
Entity:  daily 	Category:  DateTime 	Confidence Score:  0.8


## Keyword Search

In [14]:
from fuzzysearch import find_near_matches

In [20]:
find_near_matches('PATTERN', '---PATERN--- PATTERN pattern', max_l_dist=1)

[Match(start=3, end=9, dist=1, matched='PATERN'),
 Match(start=13, end=20, dist=0, matched='PATTERN')]

In [42]:
def find_keywords(article):
    """"""
    from fuzzysearch import find_near_matches
    keywords = ['fraud', 'claim', 'incident', 'bankruptcy', 'fine', 'bankrupt',
                'Fraud', 'Claim', 'Incident', 'Bankruptcy', 'Fine', 'Bankrupt']
    words = ''
    for keyword in keywords:
        count = len(find_near_matches(keyword, article, max_l_dist=0))
        if(count>0):
            words += str(count)+'x '+keyword+', '
    if(len(words)>1):
        words = words[:-2]
    return words

In [39]:
len( find_near_matches('fin', 'fineland', max_l_dist=0))

1

## Overall Sentiment

In [5]:
# Overall Sentiment  
sentiment_list = [*dff.Title_Sentiment] + [*dff.Article_Sentiment]
sentiment = max(set(sentiment_list), key = sentiment_list.count) 
message += 'The overall sentiment of the articles was '+sentiment + '.\n\n'
# Mean sentiment
def str_mean(x):
    return str(round(x.mean(),2))

message += 'Mean Sentiment: Title ('\
                            + str_mean(dff.Title_Positv)+', '\
                            + str_mean(dff.Title_Neutral)+', '\
                            + str_mean(dff.Title_Negativ)+'); '\
                        +'Article ('+str_mean(dff.Article_Positv)+', '\
                                    +str_mean(dff.Article_Positv)+', '\
                                    +str_mean(dff.Article_Negativ)+').+\n\n'
# Negativ Articles
# Articles with a negativ score >0.33
neg_th =  0.33
neg_df = dff[(dff.Title_Negativ>neg_th)|(dff.Title_Negativ>neg_th)]
neg_count=len(neg_df) 
if(neg_count==1):
    message += 'There is '+str(neg_count) + ' articles with a negativ sentiment:\n'
elif(neg_count>1):
    message += 'There are '+str(neg_count) + ' articles with a negativ sentiment:\n'
for row in neg_df.itertuples():
     message += '\t'+row.Title + ' ['+row.Title_Sentiment+' title ('\
                                        +str(row.Title_Positv)+', '\
                                        +str(row.Title_Neutral)+', '\
                                        +str(row.Title_Negativ)+'); '\
                                    +row.Article_Sentiment+' article ('\
                                        +str(row.Article_Positv)+', '\
                                        +str(row.Article_Neutral)+', '\
                                        +str(row.Article_Negativ)+')]+\n\n'

message

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [97]:
# # Overall Sentiment  
# message=''
# # Overall Sentiment  
# sentiment_list = [*df.Title_Sentiment] + [*df.Article_Sentiment]
# sentiment = max(set(sentiment_list), key = sentiment_list.count) 
# message += 'The overall sentiment of the articles was '+sentiment + '.\n\n'
# # Mean sentiment
# def str_mean(x):
#     return str(round(x.mean(),2))
    
# message += 'Mean Sentiment: Title ('\
#                             + str_mean(df.Title_Positv)+', '\
#                             + str_mean(df.Title_Neutral)+', '\
#                             + str_mean(df.Title_Negativ)+'); '\
#                         +'Article ('+str_mean(df.Article_Positv)+', '\
#                                     +str_mean(df.Article_Positv)+', '\
#                                     +str_mean(df.Article_Negativ)+').+\n\n'
# # Negativ Articles
# # Articles with a negativ score >0.33
# neg_th =  0.33
# neg_df = df[(df.Title_Negativ>neg_th)|(df.Title_Negativ>neg_th)]
# neg_count=len(neg_df) 
# if(neg_count==1):
#     message += 'There is '+str(neg_count) + ' articles with a negativ sentiment:\n'
# elif(neg_count>1):
#     message += 'There are '+str(neg_count) + ' articles with a negativ sentiment:\n'
# for row in neg_df.itertuples():
#      message += '\t'+row.Title + ' ['+row.Title_Sentiment+' title ('\
#                                         +str(row.Title_Positv)+', '\
#                                         +str(row.Title_Neutral)+', '\
#                                         +str(row.Title_Negativ)+'); '\
#                                     +row.Article_Sentiment+' article ('\
#                                         +str(row.Article_Positv)+', '\
#                                         +str(row.Article_Neutral)+', '\
#                                         +str(row.Article_Negativ)+')]+\n\n'



# # import pprint
# pp.pprint(message)

('The overall sentiment of the articles was neutral.\n'
 '\n'
 'Mean Sentiment: Title (0.09, 0.85, 0.06); Article (0.22, 0.22, 0.27).+\n'
 '\n'
 'There is 1 articles with a negativ sentiment:\n'
 '\tCritical Contrast: Deutsche Börse (DBOEY) and Its Rivals [neutral title '
 '(0.07, 0.58, 0.35); mixed article (0.33, 0.25, 0.42)]+\n'
 '\n')


## FT News

### Test ft

In [28]:
import requests
from pprint import pprint

apikey = '59cbaf20e3e06d3565778e7b7e93393d34f24330818d4de7f89b27a1'
timestamp = "2021-01-01T00:00:00.000Z"
#
query_url = f"https://api.ft.com/content/notifications?apiKey={apikey}&since={timestamp}"

r = requests.get(query_url)
pprint(r.json())

{'links': [{'href': 'https://api.ft.com/content/notifications?page=MjAyMTAxMDQyMzIxMjYuNzc0IzAjdXBwLXByb2QtZGVsaXZlcnktZXU%3D&since=2021-01-04T23%3A21%3A26.774Z',
            'rel': 'next'}],
 'notifications': [{'apiUrl': 'https://api.ft.com/content/6ed46c9b-ce04-48c3-aa38-68efbc4eebb9',
                    'id': 'http://www.ft.com/thing/6ed46c9b-ce04-48c3-aa38-68efbc4eebb9',
                    'type': 'http://www.ft.com/thing/ThingChangeType/UPDATE'},
                   {'apiUrl': 'https://api.ft.com/content/deb46ed1-b6c3-4e57-9054-1dca16dadad0',
                    'id': 'http://www.ft.com/thing/deb46ed1-b6c3-4e57-9054-1dca16dadad0',
                    'type': 'http://www.ft.com/thing/ThingChangeType/UPDATE'},
                   {'apiUrl': 'https://api.ft.com/content/d9f91804-105c-4e17-b88a-3d1017a6ba0e',
                    'id': 'http://www.ft.com/thing/d9f91804-105c-4e17-b88a-3d1017a6ba0e',
                    'type': 'http://www.ft.com/thing/ThingChangeType/UPDATE'},
          

                   {'apiUrl': 'https://api.ft.com/content/a6275216-bce4-4a5a-a582-3a99a6042732',
                    'id': 'http://www.ft.com/thing/a6275216-bce4-4a5a-a582-3a99a6042732',
                    'type': 'http://www.ft.com/thing/ThingChangeType/UPDATE'},
                   {'apiUrl': 'https://api.ft.com/content/5c2d1a1e-f783-43bd-a7ab-bb34f663309d',
                    'id': 'http://www.ft.com/thing/5c2d1a1e-f783-43bd-a7ab-bb34f663309d',
                    'type': 'http://www.ft.com/thing/ThingChangeType/UPDATE'},
                   {'apiUrl': 'https://api.ft.com/content/644035a3-fc42-4720-b414-8a9c3457799a',
                    'id': 'http://www.ft.com/thing/644035a3-fc42-4720-b414-8a9c3457799a',
                    'type': 'http://www.ft.com/thing/ThingChangeType/UPDATE'},
                   {'apiUrl': 'https://api.ft.com/content/15ddd567-9a78-472c-959b-0f54dafbbbd1',
                    'id': 'http://www.ft.com/thing/15ddd567-9a78-472c-959b-0f54dafbbbd1',
               

In [35]:
apikey = '59cbaf20e3e06d3565778e7b7e93393d34f24330818d4de7f89b27a1'
timestamp = "2021-01-01T00:00:00.000Z"
content='0be503b0-ff9e-11de-921f-00144feabdc0'
#
query_url = f"https://api.ft.com/content/{content}&apiKey={apikey}"

r = requests.get(query_url)
pprint(r.json())

{'error': 'Access to this API has been disallowed'}


In [32]:
apikey = '59cbaf20e3e06d3565778e7b7e93393d34f24330818d4de7f89b27a1'
timestamp = "2021-01-01T00:00:00.000Z"
#
query_url = f"https://api.ft.com/content/7348edd8-0403-11de-845b-000077b07658?apiKey={apikey}"

r = requests.get(query_url)
pprint(r.json())

{'error': 'Access to this API has been disallowed'}


In [76]:
apikey = '59cbaf20e3e06d3565778e7b7e93393d34f24330818d4de7f89b27a1'

query_url = f"https://api.ft.com/content/items/v1/0be503b0-ff9e-11de-921f-00144feabdc0?apiKey={apikey}"

r = requests.get(query_url)
pprint(r.json())

{'error': 'Access to this API has been disallowed'}


In [73]:
#  Retrieve all published pages
apikey = '59cbaf20e3e06d3565778e7b7e93393d34f24330818d4de7f89b27a1'

query_url = f"https://api.ft.com/site/v1/pages?apiKey={apikey}"

r = requests.get(query_url)
pprint(r.json())

JSONDecodeError: Extra data: line 1 column 5 (char 4)

### Test Rapid API

- https://rapidapi.com/blog/rapidapi-featured-news-apis/

In [ ]:
import requests

url = "https://financialtimesmikilior1v1.p.rapidapi.com/getAspectsList"

payload = "apiKey=%3CREQUIRED%3E"
headers = {
    'content-type': "application/x-www-form-urlencoded",
    'x-rapidapi-key': "59cbaf20e3e06d3565778e7b7e93393d34f24330818d4de7f89b27a1",
    'x-rapidapi-host': "FinancialTimesmikilior1V1.p.rapidapi.com"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

### Test Currents

- https://currentsapi.services/en/profile
- https://towardsdatascience.com/scraping-news-and-articles-from-public-apis-with-python-be84521d85b9

In [44]:
# https://currentsapi.services/en/docs/search
apikey = 'Zm0T2sOpC2G9LInaOGnXBdDwU29-U__Z_rXixLa-dJQdaUb2'
category = "business"
language = 'en'  # Mapping from Language to Code, e.g.: "English": "en"
country = "CA"  # Mapping from Country to Code, e.g.: "Canada": "CA",
keywords = "bitcoin"
t = "1"  # 1 for news, 2 for article and 3 for discussion content
domain = "financialpost.com"  # website primary domain name (without www or blog prefix)
start_date = "2020-06-01T14:30"  # YYYY-MM-DDTHH:MM:SS+00:00
query_url = f"https://api.currentsapi.services/v1/search?" \
            f"apiKey={apikey}" \
            f"&language={language}" \
            f"&category={category}" \
            f"&country={country}" \
            f"&type={t}" \
            f"&domain={domain}" \
            f"&keywords={keywords}" \
            f"&start_date={start_date}"

r = requests.get(query_url)
pprint(r.json())

{'news': [{'author': 'Reuters',
           'category': ['business'],
           'description': 'Digital currency Bitcoin extended its record '
                          'smashing rally on Saturday, beginning the year with '
                          'a surge over $30,000 for the first time, with ever '
                          'more traders and investors betting that it is on '
                          'its way to becoming a mainstream payment method. '
                          'Read More...',
           'id': '7d1e2c4e-886f-466f-b3f2-77c107dfe3e2',
           'image': 'None',
           'language': 'en',
           'published': '2021-01-02 15:17:04 +0000',
           'title': 'Bitcoin rallies above $30,000 for first time',
           'url': 'https://financialpost.com/pmn/business-pmn/bitcoin-rallies-above-30000-for-first-time'}],
 'page': 1,
 'status': 'ok'}


In [69]:
# https://currentsapi.services/en/docs/search
apikey = 'Zm0T2sOpC2G9LInaOGnXBdDwU29-U__Z_rXixLa-dJQdaUb2'
language = 'en'  # Mapping from Language to Code, e.g.: "English": "en"
keywords = "nasdaq"
domain = "ft.com"  # website primary domain name (without www or blog prefix)
t = "2"  # 1 for news, 2 for article and 3 for discussion content
start_date = "2021-01-01T00:00"  # YYYY-MM-DDTHH:MM:SS+00:00
query_url = f"https://api.currentsapi.services/v1/search?" \
            f"apiKey={apikey}" \
            f"&language={language}" \
            f"&type={t}" \
            f"&domain={domain}" \
            f"&keywords={keywords}" \
            f"&start_date={start_date}"

r = requests.get(query_url)
pprint(r.json())

{'news': [], 'page': 1, 'status': 'ok'}


In [65]:
query_url

'https://api.currentsapi.services/v1/search?apiKey=Zm0T2sOpC2G9LInaOGnXBdDwU29-U__Z_rXixLa-dJQdaUb2&language=en&type=2&keywords="LCH SA"&start_date=2021-01-01T00:00'

## Bing Websearch

### Final Function

In [ ]:
def get_newssearch_client():
    """"""
    # Import Libraries
    from azure.cognitiveservices.search.newssearch import NewsSearchClient
    from msrest.authentication import CognitiveServicesCredentials
    
    # Credentials
    subscription_key = "5521444b482f4dd6a800e85fdfb1c636"
    ENDPOINT = "https://api.bing.microsoft.com/v7.0/search?q=microsoft+devices"
    
    # Client
    bing_client = NewsSearchClient(
            endpoint=ENDPOINT,
            credentials=CognitiveServicesCredentials(subscription_key)
        )
    
    return bing_client

def add_bing_news(dff, searchword):
    """"""
    # Authenticate connection to azure bing news search service
    bing_client = get_newssearch_client()
    
    # Fetch News
    news_result = bing_client.news.search(query=searchword,
            #market="en-uk", NOT WORKING 
            freshness="Week",
            #setLang:"en", NOT WORKING
            sort_by="Date"
        )
    
    # Add Bing news to df  
    dff =  bing_news_to_df(news_result.as_dict().get("value"), dff)
    
    return dff

In [4]:
searchword

'LCH SA'

In [18]:
# Authenticate connection to azure bing news search service
bing_client = get_newssearch_client()

# Fetch News
news_result = bing_client.news.search(query='LCH SA',
        #market="en", #NOT WORKING 
        freshness="Week",
        #setLang:"en", NOT WORKING
        sort_by="Date"
    )

In [19]:
news_result.as_dict().get("value")

[{'_type': 'NewsArticle',
  'name': 'Matthew Stafford avait sa liste d’équipes',
  'url': 'https://www.tvasports.ca/2021/02/14/matthew-stafford-avait-sa-liste-dequipes-1',
  'image': {'_type': 'ImageObject',
   'thumbnail': {'_type': 'ImageObject',
    'content_url': 'https://www.bing.com/th?id=OVFT.9robraYI5-1eVXTjr3ytmC&pid=News',
    'width': 700,
    'height': 393}},
  'description': 'Le quart-arrière Matthew Stafford n’a pas confirmé qu’il ne voulait rien savoir d’être échangé aux Patriots de la Nouvelle-Angleterre, mais c’est comme s’il avait m',
  'provider': [{'_type': 'Organization', 'name': 'TVA Sports'}],
  'date_published': '2021-02-14T20:45:00.0000000Z'},
 {'_type': 'NewsArticle',
  'name': 'L’art de se faire un prénom',
  'url': 'https://www.msn.com/fr-ca/actualites/r%C3%A9gion%20de%20trois-rivi%C3%A8res/lart-de-se-faire-un-pr%C3%A9nom/ar-BB1dEiS9',
  'image': {'_type': 'ImageObject',
   'thumbnail': {'_type': 'ImageObject',
    'content_url': 'https://www.bing.com/th?id=

### Azure API

In [23]:
import requests

In [157]:
# Archive

# https://docs.microsoft.com/en-us/azure/cognitive-services/bing-web-search/quickstarts/client-libraries?pivots=programming-language-python

# # Import required modules.
# from azure.cognitiveservices.search.websearch import WebSearchClient
# from azure.cognitiveservices.search.websearch.models import SafeSearch
# from msrest.authentication import CognitiveServicesCredentials

- https://docs.microsoft.com/en-us/azure/cognitive-services/bing-web-search/quickstarts/python

In [ ]:
# # Working Copy
# subscription_key = "5521444b482f4dd6a800e85fdfb1c636"
# assert subscription_key

# search_url = "https://api.bing.microsoft.com/v7.0/search"

# search_term = "EUREX"

# # Search
# headers = {"Ocp-Apim-Subscription-Key": subscription_key}
# params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
# response = requests.get(search_url, headers=headers, params=params)
# response.raise_for_status()
# search_results = response.json()
# search_results

In [21]:
subscription_key = "5521444b482f4dd6a800e85fdfb1c636"
assert subscription_key

search_url = "https://api.bing.microsoft.com/v7.0/news/search"

search_term = "EUREX"
search_term = "LCH SA"

In [24]:
# Search
headers = {"Ocp-Apim-Subscription-Key": subscription_key}
params = {"q": search_term, "textDecorations": True, 
          "textFormat": "HTML",
          #"freshness":"week",
          "responseFilter":"News",
          "setLang":"en"
         }
response = requests.get(search_url, headers=headers, params=params)
response.raise_for_status()
search_results = response.json()
search_results

{'_type': 'News',
 'readLink': 'https://api.bing.microsoft.com/api/v7/news/search?q=LCH+SA&setLang=en',
 'queryContext': {'originalQuery': 'LCH SA', 'adultIntent': False},
 'totalEstimatedMatches': 19,
 'sort': [{'name': 'Beste overeenkomst',
   'id': 'relevance',
   'isSelected': True,
   'url': 'https://api.bing.microsoft.com/api/v7/news/search?q=LCH+SA&setLang=en&setLang=en'},
  {'name': 'Meest recent',
   'id': 'date',
   'isSelected': False,
   'url': 'https://api.bing.microsoft.com/api/v7/news/search?q=LCH+SA&setLang=en&sortby=date&setLang=en'}],
 'value': [{'name': 'Matthew Stafford avait <b>sa</b> liste d’équipes',
   'url': 'https://www.tvasports.ca/2021/02/14/matthew-stafford-avait-sa-liste-dequipes-1',
   'image': {'thumbnail': {'contentUrl': 'https://www.bing.com/th?id=OVFT.9robraYI5-1eVXTjr3ytmC&pid=News',
     'width': 700,
     'height': 393}},
   'description': 'Le quart-arrière Matthew Stafford n’a pas confirmé qu’il ne voulait rien savoir d’être échangé aux Patriots d

In [26]:
search_results.get("value")[0]

{'name': 'Matthew Stafford avait <b>sa</b> liste d’équipes',
 'url': 'https://www.tvasports.ca/2021/02/14/matthew-stafford-avait-sa-liste-dequipes-1',
 'image': {'thumbnail': {'contentUrl': 'https://www.bing.com/th?id=OVFT.9robraYI5-1eVXTjr3ytmC&pid=News',
   'width': 700,
   'height': 393}},
 'description': 'Le quart-arrière Matthew Stafford n’a pas confirmé qu’il ne voulait rien savoir d’être échangé aux Patriots de la Nouvelle-Angleterre, mais c’est comme s’il avait m',
 'about': [{'readLink': 'https://api.bing.microsoft.com/api/v7/entities/fb242e14-182b-68a7-12da-68d8456f512f?setLang=en',
   'name': 'Matthew Stafford'}],
 'provider': [{'_type': 'Organization', 'name': 'TVA Sports'}],
 'datePublished': '2021-02-14T20:45:00.0000000Z'}

In [27]:
# Extract articles 
from newspaper import Article
for result in search_results.get("value")[1:]:
    print(result.get("name"))
    print(result.get("url"))
    print(result.get("datePublished"))
    print()
    # Download Article
    article = Article(result.get("url"))
    article.download()
    article.parse()
    article.nlp()

    title = article.title
    text = article.text
    summary = article.summary
    date = article.publish_date

Selon David Ortiz, le baseball est devenu ennuyeux
https://www.tvasports.ca/2021/02/14/selon-david-ortiz-le-baseball-est-devenu-ennuyeux
2021-02-14T22:47:00.0000000Z

Legazpi City Hospital placed on lockdown due to Covid-19
https://www.pna.gov.ph/articles/1129633
2021-02-04T10:09:00.0000000Z

These hedge funds earned $63.5 billion in 2020
https://www.iol.co.za/personal-finance/investments/these-hedge-funds-earned-635-billion-in-2020-21c01781-528f-4e98-b062-da31720ec774
2021-01-25T17:02:00.0000000Z

Halluin: le Cyclo-club halluinois a redécouvert la Deûle, les Weppes et la Pévèle
https://www.lavoixdunord.fr/934464/article/2021-02-11/halluin-le-cyclo-club-halluinois-redecouvert-la-deule-les-weppes-et-la-pevele
2021-02-11T16:09:00.0000000Z

L’art de se faire un prénom
https://www.msn.com/fr-ca/actualites/r%C3%A9gion%20de%20trois-rivi%C3%A8res/lart-de-se-faire-un-pr%C3%A9nom/ar-BB1dEiS9
2021-02-13T08:00:00.0000000Z

Brexit : Pourquoi l’exode vers Paris ne s&#39;est pas (encore ?) réalisé
h

In [28]:
# Download Article
url = "https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/"
url = "https://www.modernreaders.com/news/2021/02/09/critical-contrast-deutsche-borse-dboey-and-its-rivals.html"
article = Article(url)
article.download()
article.parse()
article.nlp()

title = article.title
text = article.text
summary = article.summary
date = article.publish_date

### Azure Bing News API 

In [224]:
from azure.cognitiveservices.search.newssearch import NewsSearchClient
from msrest.authentication import CognitiveServicesCredentials

In [261]:
# Add your Bing Search V7 subscription key to your environment variables.
SUBSCRIPTION_KEY = "5521444b482f4dd6a800e85fdfb1c636"
# Note: your endpoint should not include the /bing/v7.0 suffix
ENDPOINT = "https://api.bing.microsoft.com/v7.0/search"
#ENDPOINT = "https://api.bing.microsoft.com/"
#ENDPOINT = "https://api.bing.microsoft.com/bing/v7.0"
ENDPOINT = "https://api.bing.microsoft.com/v7.0/search?q=microsoft+devices"

In [278]:
client = NewsSearchClient(
            endpoint=ENDPOINT,
            credentials=CognitiveServicesCredentials(subscription_key)
        )

2021-02-11 09:28:23 [msrest.universal_http.requests] DEBUG: Configuring retry: max_retries=3, backoff_factor=0.8, max_backoff=90


In [282]:
news_result = client.news.search(query="EUREX",
            #market="en-uk",
            freshness="Week",
            #setLang:"en",
            sort_by="Date"
        )

if news_result.value:
    first_news_result = news_result.value[0]
    print("Total estimated matches value: {}".format(
                                        news_result.total_estimated_matches))
    print("News result count: {}".format(len(news_result.value)))
    print("First news name: {}".format(first_news_result.name))
    print("First news url: {}".format(first_news_result.url))
    print("First news description: {}".format(
                                        first_news_result.description))
    print("First published time: {}".format(
                                        first_news_result.date_published))
    print("First news provider: {}".format(
                                        first_news_result.provider[0].name))
else:
    print("Didn't see any news result data..")

2021-02-11 09:41:32 [msrest.universal_http] DEBUG: Configuring redirects: allow=True, max=30
2021-02-11 09:41:32 [msrest.universal_http] DEBUG: Configuring request: timeout=100, verify=True, cert=None
2021-02-11 09:41:32 [msrest.universal_http] DEBUG: Configuring proxies: ''
2021-02-11 09:41:32 [msrest.universal_http] DEBUG: Evaluate proxies against ENV settings: True


Total estimated matches value: 8
News result count: 8
First news name: Clearstream Reports €15.4 Trillion in Assets Under Custody
First news url: https://www.financemagnates.com/institutional-forex/exchanges/clearstream-reports-e15-4-trillion-in-assets-under-custody/
First news description: During the market turbulence, Deutsche Boerse’s Xetra and Eurex platforms, as well as other platforms that cater to retail investors, have experienced difficulties, resulting in clients getting kept out of the market during sensitive times.
First published time: 2021-02-10T17:25:00.0000000Z
First news provider: Finance Magnates


In [230]:
# Functions
def news_search(searchword="Azure", subscription_key=SUBSCRIPTION_KEY):
    """NewsSearch.
    This will search news for (Quantum  Computing) with market and count parameters then verify number of results and print out totalEstimatedMatches, name, url, description, published time and name of provider of the first news result
    """
    client = NewsSearchClient(
        endpoint=ENDPOINT,
        credentials=CognitiveServicesCredentials(subscription_key)
    )

    try:
        news_result = client.news.search(
            query=searchword, market="en-us", count=10)
        print("Search news for query "+searchword+" with market and count")

        if news_result.value:
            first_news_result = news_result.value[0]
            print("Total estimated matches value: {}".format(
                news_result.total_estimated_matches))
            print("News result count: {}".format(len(news_result.value)))
            print("First news name: {}".format(first_news_result.name))
            print("First news url: {}".format(first_news_result.url))
            print("First news description: {}".format(
                first_news_result.description))
            print("First published time: {}".format(
                first_news_result.date_published))
            print("First news provider: {}".format(
                first_news_result.provider[0].name))
        else:
            print("Didn't see any news result data..")

    except Exception as err:
        print("Encountered exception. {}".format(err))

def news_search_with_filtering(searchword="Azure", subscription_key=SUBSCRIPTION_KEY):
    """NewsSearchWithFilters.
    This will search most recent news for searchword
    with freshness and sortBy parameters 
    then verify number of results 
    and print out totalEstimatedMatches, name, url, description, 
    published time and name of provider of the first news result.
    """
    client = NewsSearchClient(
        endpoint="https://api.cognitive.microsoft.com",
        credentials=CognitiveServicesCredentials(subscription_key)
    )

    try:
        news_result = client.news.search(
            query=searchword,
            market="en-us",
            freshness="Week",
            sort_by="Date"
        )
        print("Search most recent news for query \"Artificial Intelligence\" with freshness and sortBy")

        if news_result.value:
            first_news_result = news_result.value[0]
            print("News result count: {}".format(len(news_result.value)))
            print("First news name: {}".format(first_news_result.name))
            print("First news url: {}".format(first_news_result.url))
            print("First news description: {}".format(
                first_news_result.description))
            print("First published time: {}".format(
                first_news_result.date_published))
            print("First news provider: {}".format(
                first_news_result.provider[0].name))
        else:
            print("Didn't see any news result data..")

    except Exception as err:
        print("Encountered exception. {}".format(err))

def news_category(category="Entertainment_MovieAndTV", subscription_key=SUBSCRIPTION_KEY):
    """NewsCategory.
    This will search category news for movie and TV entertainment with safe search then verify number of results and print out category, name, url, description, published time and name of provider of the first news result.
    """
    client = NewsSearchClient(
        endpoint="https://api.cognitive.microsoft.com",
        credentials=CognitiveServicesCredentials(subscription_key)
    )

    try:
        news_result = client.news.category(
            category=category,
            market="en-us",
            safe_search="strict"
        )
        print("Search category news for movie and TV entertainment with safe search")

        if news_result.value:
            first_news_result = news_result.value[0]
            print("News result count: {}".format(len(news_result.value)))
            print("First news category: {}".format(first_news_result.category))
            print("First news name: {}".format(first_news_result.name))
            print("First news url: {}".format(first_news_result.url))
            print("First news description: {}".format(
                first_news_result.description))
            print("First published time: {}".format(
                first_news_result.date_published))
            print("First news provider: {}".format(
                first_news_result.provider[0].name))
        else:
            print("Didn't see any news result data..")

    except Exception as err:
        print("Encountered exception. {}".format(err))

def news_trending(subscription_key):
    """NewsTrending.
    This will search news trending topics in Bing then verify number of results and print out name, text of query, webSearchUrl, newsSearchUrl and image Url of the first news result.
    """
    client = NewsSearchClient(
        endpoint="https://api.cognitive.microsoft.com",
        credentials=CognitiveServicesCredentials(subscription_key)
    )

    try:
        trending_topics = client.news.trending(market="en-us")
        print("Search news trending topics in Bing")

        if trending_topics.value:
            first_topic = trending_topics.value[0]
            print("News result count: {}".format(len(trending_topics.value)))
            print("First topic name: {}".format(first_topic.name))
            print("First topic query: {}".format(first_topic.query.text))
            print("First topic image url: {}".format(first_topic.image.url))
            print("First topic webSearchUrl: {}".format(
                first_topic.web_search_url))
            print("First topic newsSearchUrl: {}".format(
                first_topic.news_search_url))
        else:
            print("Didn't see any topics result data..")

    except Exception as err:
        print("Encountered exception. {}".format(err))

In [291]:
news_result.as_dict().get("value")[0]

{'_type': 'NewsArticle',
 'name': 'Clearstream Reports €15.4 Trillion in Assets Under Custody',
 'url': 'https://www.financemagnates.com/institutional-forex/exchanges/clearstream-reports-e15-4-trillion-in-assets-under-custody/',
 'image': {'_type': 'ImageObject',
  'thumbnail': {'_type': 'ImageObject',
   'content_url': 'https://www.bing.com/th?id=OVFT.QV2qeWaojG1Mxo1zhu6qTS&pid=News',
   'width': 700,
   'height': 318}},
 'description': 'During the market turbulence, Deutsche Boerse’s Xetra and Eurex platforms, as well as other platforms that cater to retail investors, have experienced difficulties, resulting in clients getting kept out of the market during sensitive times.',
 'provider': [{'_type': 'Organization', 'name': 'Finance Magnates'}],
 'date_published': '2021-02-10T17:25:00.0000000Z'}

In [305]:
#result.get("description")
# result.get("provider")[0].get("name")
article = Article(result.get("url"))
article.download()
article.parse()
article.nlp()
article.

AttributeError: 'Article' object has no attribute 'language'

In [296]:
from newspaper import Article
for result in news_result.as_dict().get("value"):
    print(result.get("name"))
    print(result.get("url"))
    print(result.get("date_published"))
    try:
        # Download Article
        article = Article(result.get("url"))
        article.download()
        article.parse()
        article.nlp()

        title = article.title
        text = article.text
        summary = article.summary
        date = article.publish_date
    except:
        print("Not possible for this Url")
    print()

Clearstream Reports €15.4 Trillion in Assets Under Custody
https://www.financemagnates.com/institutional-forex/exchanges/clearstream-reports-e15-4-trillion-in-assets-under-custody/
2021-02-10T17:25:00.0000000Z

Critical Contrast: Deutsche Börse (DBOEY) and Its Rivals
https://www.modernreaders.com/news/2021/02/09/critical-contrast-deutsche-borse-dboey-and-its-rivals.html
2021-02-09T17:39:00.0000000Z

Vienna Stock Exchange: New Calculation Technology Increases Quality
https://www.vindobona.org/article/vienna-stock-exchange-new-calculation-technology-increases-quality
2021-02-09T16:01:00.0000000Z


2021-02-11 10:29:59 [newspaper.article] DEBUG: Download failed on URL https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/ because of 403 Client Error: Forbidden for url: https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/



Eurex traded contracts fall 13%MoM in January to 129.5 million
https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/
2021-02-08T10:02:00.0000000Z
Not possible for this Url

Contrasting Deutsche Börse (DBOEY) and Its Peers
https://www.modernreaders.com/news/2021/02/08/contrasting-deutsche-borse-dboey-and-its-peers.html
2021-02-08T08:03:00.0000000Z

January 2021 monthly figures at Eurex
https://www.mds.deutsche-boerse.com/dbg-en/media/press-releases/January-2021-monthly-figures-at-Eurex--2433156
2021-02-05T12:58:00.0000000Z

GFF Summit 2021: two days of pure expertise
https://www.mds.deutsche-boerse.com/dbg-en/products-services/insights-post-trading-en/GFF-Summit-2021-two-days-of-pure-expertise-2434356
2021-02-05T00:00:00.0000000Z

'Handel stroomt weg van Eurex door nieuwe regel'
https://fd.nl/beurs/1372845/handel-stroomt-weg-van-eurex-door-nieuwe-regel
2021-02-04T17:57:00.0000000Z



### Bing News Function

In [294]:
# Input
searchword = 'EUREX'
dff = pd.DataFrame()

In [319]:
def add_bing_news(dff, searchword):
    """"""
    # Import Libraries
    from azure.cognitiveservices.search.newssearch import NewsSearchClient
    from msrest.authentication import CognitiveServicesCredentials
    
    # Credentials
    SUBSCRIPTION_KEY = "5521444b482f4dd6a800e85fdfb1c636"
    ENDPOINT = "https://api.bing.microsoft.com/v7.0/search?q=microsoft+devices"
    
    # Client
    bing_client = NewsSearchClient(
            endpoint=ENDPOINT,
            credentials=CognitiveServicesCredentials(subscription_key)
        )
    
    # Fetch News
    news_result = bing_client.news.search(query=searchword,
            #market="en-uk", NOT WORKING 
            freshness="Week",
            #setLang:"en", NOT WORKING
            sort_by="Date"
        )
    

    # Authenticate connection to azure text analytics service
    ta_client = authenticate_azure_textanalytics_client()
    
    #
    for result in news_result.as_dict().get("value"):
        name = result.get("name")
        url = result.get("url")
        date = result.get("date_published")
        description = result.get("description")
        provider = result.get("provider")[0].get("name")
        
        row = {}
        
        try:
            # Download Article
            article = Article(result.get("url"))
            article.download()
            article.parse()
            article.nlp()

            title = article.title
            text = article.text
            summary = article.summary
            date = article.publish_date
            
            # Add Articles to Df 
            row = {'API':provider,
                        'Link':url, 
                        'Date':date, 
                        'Title':title, 
                        'Article':text, 
                        'Summary':summary, 
                        'Topics':'',
                        'Found_Keywords':'', 
                        'Overall_Sentiment':'', 
                        'Verified':'', 
                        'Title_Sentiment':getAzureSentiment(title, ta_client)[0],
                        'Title_Positv':getAzureSentiment(title, ta_client)[1], 
                        'Title_Neutral':getAzureSentiment(title, ta_client)[2],
                        'Title_Negativ':getAzureSentiment(title, ta_client)[3], 
                        'Article_Sentiment':getAzureSentiment(text, ta_client)[0],
                        'Article_Positv':getAzureSentiment(text, ta_client)[1], 
                        'Article_Neutral':getAzureSentiment(text, ta_client)[2], 
                        'Article_Negativ':getAzureSentiment(text, ta_client)[3]}
        except:
            print('Article could not be downloaded. ' + url)
            row = {'API':provider,
                        'Link':url, 
                        'Date':date, 
                        'Title':name, 
                        'Article':description, 
                        'Summary':'', 
                        'Topics':'',
                        'Found_Keywords':'', 
                        'Overall_Sentiment':'', 
                        'Verified':'', 
                        'Title_Sentiment':getAzureSentiment(name, ta_client)[0],
                        'Title_Positv':getAzureSentiment(name, ta_client)[1], 
                        'Title_Neutral':getAzureSentiment(name, ta_client)[2],
                        'Title_Negativ':getAzureSentiment(name, ta_client)[3], 
                        'Article_Sentiment':getAzureSentiment(description, ta_client)[0],
                        'Article_Positv':getAzureSentiment(description, ta_client)[1], 
                        'Article_Neutral':getAzureSentiment(description, ta_client)[2], 
                        'Article_Negativ':getAzureSentiment(description, ta_client)[3]}
            #continue
        
        # Add row to dff
        dff = dff.append(row, ignore_index=True)
        
    return dff 
    
    

In [321]:
x = add_bing_news(df, "EUREX")
x

2021-02-11 14:52:46 [msrest.universal_http.requests] DEBUG: Configuring retry: max_retries=3, backoff_factor=0.8, max_backoff=90
2021-02-11 14:52:46 [msrest.universal_http] DEBUG: Configuring redirects: allow=True, max=30
2021-02-11 14:52:46 [msrest.universal_http] DEBUG: Configuring request: timeout=100, verify=True, cert=None
2021-02-11 14:52:46 [msrest.universal_http] DEBUG: Configuring proxies: ''
2021-02-11 14:52:46 [msrest.universal_http] DEBUG: Evaluate proxies against ENV settings: True


Article could not be downloaded. https://www.vindobona.org/article/vienna-stock-exchange-new-calculation-technology-increases-quality


2021-02-11 14:52:56 [newspaper.article] DEBUG: Download failed on URL https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/ because of 403 Client Error: Forbidden for url: https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/


Article could not be downloaded. https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/


,API,Link,Date,Title,Article,Summary,Topics,Found_Keywords,Overall_Sentiment,Verified,Title_Sentiment,Title_Positv,Title_Neutral,Title_Negativ,Article_Sentiment,Article_Positv,Article_Neutral,Article_Negativ
0,PR Newswire,https://www.prnewswire.com/news-releases/fenic...,None,Fenics GO named OTC Trading Platform of the Year,"LONDON, Feb. 11, 2021 /PRNewswire/ -- Fenics G...","LONDON, Feb. 11, 2021 /PRNewswire/ -- Fenics G...",,,,,neutral,0.01,0.98,0.01,mixed,0.44,0.11,0.45
1,Finance Magnates,https://www.financemagnates.com/institutional-...,2021-02-10 17:10:27+00:00,Clearstream Reports €15.4 Trillion in Assets U...,Settlement volumes at Deutsche Boerse AG’s Cle...,Settlement volumes at Deutsche Boerse AG’s Cle...,,,,,neutral,0.01,0.89,0.10,mixed,0.30,0.10,0.60
2,modernreaders.com,https://www.modernreaders.com/news/2021/02/09/...,2021-02-09 00:00:00,Critical Contrast: Deutsche Börse (DBOEY) and ...,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,ProfitabilityGet Deutsche Börse alerts:This ta...,,,,,neutral,0.07,0.58,0.35,mixed,0.33,0.25,0.42
3,vindobona,https://www.vindobona.org/article/vienna-stock...,None,Vienna Stock Exchange: New Calculation Technol...,The Vienna Stock Exchange (Wiener Börse AG) ha...,,,,,,neutral,0.01,0.99,0.00,positive,0.50,0.48,0.02
4,Leaprate,https://www.leaprate.com/financial-services/ex...,2021-02-08T10:02:00.0000000Z,Eurex traded contracts fall 13%MoM in January ...,Eurex Repo posted drops 25.2% MoM rise in GC P...,,,,,,neutral,0.00,0.96,0.04,neutral,0.01,0.87,0.12
5,modernreaders.com,https://www.modernreaders.com/news/2021/02/08/...,2021-02-08 00:00:00,Contrasting Deutsche Börse (DBOEY) and Its Peers,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,DividendsGet Deutsche Börse alerts:Deutsche Bö...,,,,,neutral,0.03,0.95,0.02,mixed,0.30,0.21,0.49
6,Deutsche Börse,https://www.mds.deutsche-boerse.com/dbg-en/med...,None,January 2021 monthly figures at Eurex,The number of traded financial derivatives con...,The number of traded financial derivatives con...,,,,,neutral,0.00,0.99,0.01,neutral,0.01,0.92,0.07
7,Deutsche Börse,https://www.mds.deutsche-boerse.com/dbg-en/pro...,None,GFF Summit 2021: two days of pure expertise,Following the theme “An open eco- system for a...,Following the theme “An open eco- system for a...,,,,,positive,0.75,0.25,0.00,neutral,0.05,0.91,0.04
8,Het Financieele Dagblad,https://fd.nl/beurs/1372845/handel-stroomt-weg...,None,'Handel stroomt weg van Eurex door nieuwe regel',Heeft u een FD.nl account of bent u abonnee va...,Heeft u een FD.nl account of bent u abonnee va...,,,,,neutral,0.03,0.95,0.02,neutral,0.01,0.97,0.02


__Test__

In [12]:
# Import Libraries
from azure.cognitiveservices.search.newssearch import NewsSearchClient
from msrest.authentication import CognitiveServicesCredentials

# Credentials
subscription_key = "5521444b482f4dd6a800e85fdfb1c636"
ENDPOINT = "https://api.bing.microsoft.com/v7.0/search?q=microsoft+devices"

# Client
bing_client = NewsSearchClient(
        endpoint=ENDPOINT,
        credentials=CognitiveServicesCredentials(subscription_key)
    )

# Fetch News
news_result = bing_client.news.search(query=searchword,
        #market="en-uk", NOT WORKING 
        freshness="Week",
        #setLang:"en", NOT WORKING
        sort_by="Date"
    )


# Authenticate connection to azure text analytics service
ta_client = authenticate_azure_textanalytics_client()

2021-02-11 15:05:37 [msrest.universal_http.requests] DEBUG: Configuring retry: max_retries=3, backoff_factor=0.8, max_backoff=90
2021-02-11 15:05:37 [msrest.universal_http] DEBUG: Configuring redirects: allow=True, max=30
2021-02-11 15:05:37 [msrest.universal_http] DEBUG: Configuring request: timeout=100, verify=True, cert=None
2021-02-11 15:05:37 [msrest.universal_http] DEBUG: Configuring proxies: ''
2021-02-11 15:05:37 [msrest.universal_http] DEBUG: Evaluate proxies against ENV settings: True


In [13]:
result = news_result.as_dict().get("value")[0]

name = result.get("name")
url = result.get("url")
date = result.get("date_published")[:10]
description = result.get("description")
provider = result.get("provider")[0].get("name")

row = {}

In [14]:
article = Article(result.get("url"))
article.download()
article.parse()
article.nlp()

In [15]:
title = article.title
text = article.text
summary = article.summary
date = article.publish_date[:10]

TypeError: 'NoneType' object is not subscriptable

In [17]:
article.publish_date

In [ ]:

#
for result in news_result.as_dict().get("value"):
    name = result.get("name")
    url = result.get("url")
    date = result.get("date_published")[:10]
    description = result.get("description")
    provider = result.get("provider")[0].get("name")

    row = {}

    try:
        # Download Article
        article = Article(result.get("url"))
        article.download()
        article.parse()
        article.nlp()

        title = article.title
        text = article.text
        summary = article.summary
        date = article.publish_date[:10]

        # Add Articles to Df 
        row = {'API':'Bing',
                    'Link':url, 
                    'Date':date, 
                    'Title':title, 
                    'Article':text, 
                    'Summary':summary, 
                    'Topics':'',
                    'Found_Keywords':'', 
                    'Overall_Sentiment':'', 
                    'Verified':'', 
                    'Title_Sentiment':getAzureSentiment(title, ta_client)[0],
                    'Title_Positv':getAzureSentiment(title, ta_client)[1], 
                    'Title_Neutral':getAzureSentiment(title, ta_client)[2],
                    'Title_Negativ':getAzureSentiment(title, ta_client)[3], 
                    'Article_Sentiment':getAzureSentiment(text, ta_client)[0],
                    'Article_Positv':getAzureSentiment(text, ta_client)[1], 
                    'Article_Neutral':getAzureSentiment(text, ta_client)[2], 
                    'Article_Negativ':getAzureSentiment(text, ta_client)[3]}
    except:
        #print('Article could not be downloaded. ' + url)
        row = {'API':'Bing',
                    'Link':url, 
                    'Date':date, 
                    'Title':name, 
                    'Article':description, 
                    'Summary':'xxx', 
                    'Topics':'',
                    'Found_Keywords':'', 
                    'Overall_Sentiment':'', 
                    'Verified':'', 
                    'Title_Sentiment':getAzureSentiment(name, ta_client)[0],
                    'Title_Positv':getAzureSentiment(name, ta_client)[1], 
                    'Title_Neutral':getAzureSentiment(name, ta_client)[2],
                    'Title_Negativ':getAzureSentiment(name, ta_client)[3], 
                    'Article_Sentiment':getAzureSentiment(description, ta_client)[0],
                    'Article_Positv':getAzureSentiment(description, ta_client)[1], 
                    'Article_Neutral':getAzureSentiment(description, ta_client)[2], 
                    'Article_Negativ':getAzureSentiment(description, ta_client)[3]}
        #continue

    # Add row to dff
    dff = dff.append(row, ignore_index=True)